In [ ]:
# !pip install torch torchaudio torchvision

In [3]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### 1 .Download Dataset
### 2 .Create Data Loader
### 3 .Build Model
### 4 .Train
### 5 .Save Train Model

In [4]:
# Downlaod data
def download_mnist_datasets():
  train_data = datasets.MNIST(
      root='data',
      download = True,
      train= True,
      transform = ToTensor()
  )

  validation_data = datasets.MNIST(
      root='data',
      download = True,
      train= False,
      transform = ToTensor()
  )

  return train_data, validation_data
  

In [ ]:
# download dataset
train_data, _ = download_mnist_datasets()
print("Dataset downloaded")

In [6]:
# create a dataloader (class used to wrap a dataset
                        # it's allows to load data into batches
                      #)

# for train 
BATCH_SIZE = 128
train_data_loader = DataLoader(train_data,
                               batch_size = BATCH_SIZE)



In [7]:
# Build the model
class FeedForwardNet(nn.Module):
  
  def __init__(self):
    super().__init__()
    
    self.flatten = nn.Flatten()
    self.dense_layers = nn.Sequential(
        nn.Linear(28*28, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )
    self.softmax = nn.Softmax(dim=1)

  def forward(self,input_data):
    
    flattened_data = self.flatten(input_data)
    logits = self.dense_layers(flattened_data)
    predictions = self.softmax(logits)
    return logits  


In [8]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

print(f"using device {device}")

using device cuda


In [9]:
# build the model
feed_forward_net = FeedForwardNet().to(device)

In [10]:
# train to model
def train_one_epoch(model, data_loader, loss_fn, optimiser, device):
  for inputs, targets in data_loader:
    inputs, targets = inputs.to(device), targets.to(device)
    
    # calculate loss
    predictions = model(inputs)
    loss = loss_fn(predictions, targets)

    # backpropagate loss and update weights
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
  
  print(f"Loss : {loss.item()}")


In [12]:
def train(model, data_loader, loss_fn, optimiser, device, epochs):
  for i in range(epochs):
    print(f"Epoch {i+1}")
    train_one_epoch(model, data_loader, loss_fn, optimiser, device)
    print("---------------------------")
  
  print("Training is Done!")

In [13]:
# loss fun + optimizer
LEARNING_RATE = .001

loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(
    feed_forward_net.parameters(),
    lr=LEARNING_RATE
)

In [14]:
# train the model
EPOCHS = 10
train(feed_forward_net, train_data_loader, loss_fn, optimiser, device, EPOCHS)

Epoch 1
Loss : 0.2623773515224457
---------------------------
Epoch 2
Loss : 0.21965976059436798
---------------------------
Epoch 3
Loss : 0.1884404420852661
---------------------------
Epoch 4
Loss : 0.1649537831544876
---------------------------
Epoch 5
Loss : 0.13509759306907654
---------------------------
Epoch 6
Loss : 0.11058839410543442
---------------------------
Epoch 7
Loss : 0.08938881009817123
---------------------------
Epoch 8
Loss : 0.0700596496462822
---------------------------
Epoch 9
Loss : 0.04868432506918907
---------------------------
Epoch 10
Loss : 0.0280377846211195
---------------------------
Training is Done!


In [15]:
# save the model
torch.save(feed_forward_net.state_dict(), 
           "feedforwardnet.pth")
print("Model saved")

Model saved
